# Comparison of Different Model Types
This notebook compares the performance of MLP, CNN, ViT and Hybrid models on MNIST, CIFAR10 and CIFAR100.

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR
from vit_pytorch import ViT
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from timer import Timer
import numpy as np
from sklearn import metrics
%matplotlib inline

2025-04-24 07:00:08.901141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745503208.916202 1236403 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745503208.920805 1236403 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 07:00:08.938273: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def count_parameters(model):
    """ Counts the number of trainable parameters

    Args:
        model (object): A PyTorch model.

    Returns:
        int: The number of trainable parameters
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
# Define a transform to convert grayscale to RGB
class GrayToRGB(object):
    def __call__(self, image):
        return image.repeat(3, 1, 1)

In [4]:

class MLPNet(nn.Module):
    """ Creates a fully connected multilayer perceptron. Derived from the PyTorch nn.Module class.
    """

    def __init__(self, image_size, num_classes, channels):
        """ Initializes the model

        Args:
            image_size (array): The resolution of the image
            num_classes (int): The number of output classes
            channels (int): The color depth of the image
        """
        super(MLPNet, self).__init__()
        
        self.fc1 = nn.Linear(image_size * channels, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        """ Performs inference on an image

        Args:
            x (array): The image

        Returns:
            array: The prediction distribution
        """
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)

        output = F.log_softmax(x, dim=1)
        return output  

In [5]:
class CNNNet(nn.Module):
    """ Creates a convolutional neural network. Derived from the PyTorch nn.Module class.
    """

    def __init__(self, image_size, num_classes, channels, ):
        """ Initializes the model

        Args:
            image_size (tuple): The height and width of the image
            num_classes (int): The number of output classes
            channels (int): The color depth of the image
        """
        
        super(CNNNet, self).__init__()

        (image_height, image_width) = image_size
        self.dropout = nn.Dropout(0.25)
        
        self.norm1 = nn.BatchNorm2d(32)
        self.norm2 = nn.BatchNorm2d(64)
        self.norm3 = nn.BatchNorm2d(64)
        
        self.conv1 = nn.Conv2d(channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)

        fc_size = self.forward(torch.rand((1, channels, image_height, image_width)), True)

        self.fc1 = nn.Linear(fc_size, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x, get_fc_size=False):
        """ Performs inference on an image

        Args:
            x (array): The image
            get_fc_size (bool): Flag that determines the flattened size of the last convolutional layer

        Returns:
            array: The prediction distribution
        """
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = torch.flatten(x, 1)

        if get_fc_size:
            return x.nelement()

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        output = F.log_softmax(x, dim=1)
        return output

In [6]:
class ViTNet(nn.Module): 
    """ Creates a vision transformer. Derived from the PyTorch nn.Module class.
    """
    
    def __init__(self, image_size, patch_size, num_classes, channels):
        """ Initializes the model

        Args:
            image_size (array): The resolution of the image
            patch_size (int): The size (patch_size x patch_size) of the patches to extract from the image 
            num_classes (int): The number of output classes
            channels (int): The color depth of the image
        """
        HIDDEN_SIZE = 768
        DEPTH = 6
        HEADS = 12
        MLP_DIM = 3072
        
        super(ViTNet, self).__init__()
        
        self.model = ViT(
                        image_size = image_size,
                        patch_size = patch_size,
                        num_classes = num_classes,
                        dim = HIDDEN_SIZE,
                        depth = DEPTH,
                        heads = HEADS,
                        mlp_dim = MLP_DIM,
                        channels = channels,
                        dropout = 0.1,
                        emb_dropout = 0.1
                        )

    def forward(self, x):
        """ Performs inference on an image

        Args:
            x (array): The image

        Returns:
            array: The prediction distribution
        """
        x = self.model(x)
        output = F.log_softmax(x, dim=1)
        return output

In [7]:
class HybridNet(nn.Module):
    """ Creates a CNN connected to a ViT. Derived from the PyTorch nn.Module class.
    """

    def __init__(self, image_size, channels, num_classes):
        """ Initializes the model

        Args:
            image_size (array): The resolution of the image
            channels (int): The color depth of the image
            num_classes (int): The number of output classes
        """
        HIDDEN_SIZE = 768
        DEPTH = 6
        HEADS = 12
        MLP_DIM = 3072

        super(HybridNet, self).__init__()
        
        self.conv1 = nn.Conv2d(channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        self.norm1 = nn.BatchNorm2d(32)
        self.norm2 = nn.BatchNorm2d(64)

        self.dropout = nn.Dropout(0.25)

        image_width = int((math.sqrt(image_size) - 4) / 2)

        self.vit = ViT(
                        image_size = image_width ** 2,
                        patch_size = int(image_width / 2),
                        num_classes = num_classes,
                        dim = HIDDEN_SIZE,
                        depth = DEPTH,
                        heads = HEADS,
                        mlp_dim = MLP_DIM,
                        channels = 64,
                        dropout = 0.1,
                        emb_dropout = 0.1
                        )

    def forward(self, x):
        """ Performs inference on an image

        Args:
            x (array): The image

        Returns:
            array: The prediction distribution
        """
        x = self.conv1(x)
        #x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.norm1(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.norm2(x)
        x = self.dropout(x)
        x = self.vit(x)

        output = F.log_softmax(x, dim=1)
        return output

In [8]:

class DenseNet(nn.Module):
    """ Creates a DenseNet backbone with a new classification head. Derived from the PyTorch nn.Module class.
    """

    def __init__(self, image_size, channels, num_classes):
        """ Initializes the model

        Args:
            image_size (array): The resolution of the image
            channels (int): The color depth of the image
            num_classes (int): The number of output classes
        """
        super(DenseNet, self).__init__()
        
        self.densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
        num_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Linear(num_features, num_classes) # replace num_classes with your desired number of output classes

    def forward(self, x):
        """ Performs inference on an image

        Args:
            x (array): The image

        Returns:
            array: The prediction distribution
        """
        #x = self.densenet.features(x)
        x = self.densenet(x)
        output = F.log_softmax(x, dim=1)
        return output


In [9]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    """ Performs a single iteration of training

    Args:
        model (object): The model to train
        device (object): The device that will host the training
        train_loader (object): The training dataset
        optimizer (object): The optimization algorithm
        epoch (int): The current epoch
        log_interval (int): The frequency to log data to the screen
    """
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [10]:
def test(model, device, test_loader, dataset, model_type, writer, epoch):
    """_summary_

    Args:
        model (object): The model to test
        device (object): The device that will host the testing
        test_loader (object): The test dataset
        dataset (string): The name of the dataset to use in testing
        model_type (string): The type of the model to test
        writer (object): Tensorboard logger
        epoch (int): The current epoch in training

    Returns:
        accuracy (): 
        actual ():
        predicted (): 
    """
    model.eval()
    test_loss = 0
    correct = 0
    actual = []
    predicted = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            with Timer("{} {}".format(model_type, dataset)):
                output = model(data)

            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            actual.extend(target.cpu().numpy())
            predicted.extend([np.argmax(pred) for pred in output.cpu().numpy()])

    test_loss /= len(test_loader.dataset)

    accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    writer.add_scalar('Accuracy/{} {}'.format(model_type, dataset), accuracy, epoch)
    return accuracy, actual, predicted

In [11]:
# The models and datasets that we'll be evaluating

MODEL_TYPES = ["MLP", "CNN", "ViT", "HYBRID", "DenseNet"]
DATASETS    = ["MNIST", "CIFAR10", "CIFAR100"] 
#MODEL_TYPES = ["DenseNet"]
#DATASETS    = ["MNIST"] 

In [12]:
def run(batch_size, test_batch_size, lr, gamma, epochs, log_dir, log_interval):
    """ Performs training and testing for all models and datasets specified above 

    Args:
        batch_size (int): The training batch size
        test_batch_size (int): The test batch size
        lr (float): The learning rate
        gamma (float): _description_
        epochs (int): The number of epochs of training
        log_dir (_type_): The directory for the logs
        log_interval (int): How frequently to log to the screen

    Returns:
        accuracies (dictionary): The model accuracies by model type and dataset
        parameters (dictionary): The number of trainable parameters in each model
        expected_vs_actual (dictionary): The expected and actual values used in testing each of the models
    """
    # Determine if we should use a GPU
    use_cuda = torch.cuda.is_available()

    # Reduce precision to speed up training
    torch.set_float32_matmul_precision('medium')

    if use_cuda:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs  = {'batch_size': test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    accuracies = {}
    parameters = {}
    expected_vs_actual = {}
    
    for dataset in DATASETS:
        for model_type in MODEL_TYPES:
            print("Evaluating {} with {}".format(model_type, dataset))

            if dataset == "MNIST":
                if model_type == "DenseNet":
                    transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307), (0.3081)),
                    transforms.Resize((224, 224)),
                    GrayToRGB(),
                ])
                else:
                    transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307), (0.3081))
                ])
                train_set = datasets.MNIST('./data', train=True, download=True, transform=transform)
                test_set = datasets.MNIST('./data', train=False, transform=transform)
                image_height = 28
                image_width = 28
                patch_size = 14
                num_classes = 10
                channels = 1
            elif dataset == "CIFAR10":
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                ])
                train_set = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
                test_set = datasets.CIFAR10('./data', train=False, transform=transform)
                image_height = 32
                image_width = 32
                patch_size = 16
                num_classes = 10
                channels = 3
            elif dataset == "CIFAR100":
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                ])
                train_set = datasets.CIFAR100('./data', train=True, download=True, transform=transform)
                test_set = datasets.CIFAR100('./data', train=False, transform=transform)
                image_height = 32
                image_width = 32
                patch_size = 16
                num_classes = 100
                channels = 3
                
            train_loader = torch.utils.data.DataLoader(train_set, **train_kwargs)
            test_loader  = torch.utils.data.DataLoader(test_set, **test_kwargs)

            if model_type == "MLP":
                model = MLPNet(image_height * image_width, num_classes, channels).to(device)
            elif model_type == "CNN":
                model = CNNNet((image_height, image_width), num_classes, channels).to(device)
            elif model_type == "ViT":
                model = ViTNet(image_height * image_width, patch_size, num_classes, channels).to(device)
            elif model_type == "HYBRID":
                model = HybridNet(image_height * image_width, channels, num_classes).to(device)
            elif model_type == "DenseNet":
                model = DenseNet((image_height, image_width), channels, num_classes).to(device)
            
            optimizer = optim.Adam(model.parameters(), lr=lr)
            scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

            max_accuracy = 0
            writer = SummaryWriter(log_dir=log_dir)
            for epoch in range(1, epochs + 1):
                train(model, device, train_loader, optimizer, epoch, log_interval)
                accuracy, actual, predicted = test(model, device, test_loader, dataset, model_type, writer, epoch)
                max_accuracy = max(accuracy, max_accuracy)
                scheduler.step()
                if (max_accuracy > 98):
                    print('Breaking at epoch {}'.format(epoch))
                    break

            accuracies[f'{model_type} {dataset}'] = max_accuracy
            parameters[f'{model_type} {dataset}'] = count_parameters(model)
            expected_vs_actual[f'{model_type} {dataset}'] = (actual, predicted)

            torch.save(model.state_dict(), "{}_{}.pt".format(model_type, dataset))

    return accuracies, parameters, expected_vs_actual

In [ ]:
batch_size      = 16             # Input batch size for training (default: 64)
epochs          = 20             # Number of epochs to train (default: 20)
lr              = .00001         # Learning rate (default: .00001)
gamma           = 0.7            # Learning rate step gamma (default: 0.7)
test_batch_size = 1000           # Input batch size for testing (default: 1000)
log_dir         = "./logs/tb"    # TF log dir (default: "./logs/tb")
log_interval    = 100            # Batch interval for logging (default: 100)

accuracies, parameters, expected_vs_actual = run(batch_size, test_batch_size, lr, gamma, epochs, log_dir, log_interval)

Evaluating MLP with MNIST
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.283136
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.297636
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.279965
Train Epoch: 1 [4800/60000 (8%)]	Loss: 2.251291
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.266331
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.262765
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.125573
Train Epoch: 1 [11200/60000 (19%)]	Loss: 2.091556
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.160460
Train Epoch: 1 [14400/60000 (24%)]	Loss: 2.114602
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.982369
Train Epoch: 1 [17600/60000 (29%)]	Loss: 1.908246
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.907685
Train Epoch: 1 [20800/60000 (35%)]	Loss: 1.900480
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1.836185
Train Epoch: 1 [24000/60000 (40%)]	Loss: 1.725305
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.851156
Train Epoch: 1 [27200/60000 (45%)]	Loss: 1.539961
Train Epoch: 1 [28800/60000 (48%)]	Loss: 1.215818
Train Epoch: 1 [30400/60000 (51%)]	Los

In [ ]:
# Plot the collected metrics
times = Timer.times()
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(20,10))
bar_colors = ['tab:red', 'tab:green', 'tab:blue', 'tab:orange', 'tab:purple']
ax1.bar(list(times.keys()), list(times.values()), color=bar_colors)
ax1.set_ylabel('Inference Times (ms)')
ax2.bar(list(times.keys()), list(parameters.values()), color=bar_colors)
ax2.set_ylabel('Total Parameters')
ax3.bar(list(times.keys()), list(accuracies.values()), color=bar_colors)
ax3.set_ylabel('Model Accuracy')
plt.show()

In [ ]:
# Plot the confusion matrix
for key, (actual, predicted) in expected_vs_actual.items():
    fig, (ax1) = plt.subplots(ncols=1, figsize=(5, 5))
    ax1.set_title(f'{key} Confusion Matrix')
    cm = metrics.confusion_matrix(actual, predicted)
    ax1.matshow(cm, cmap='binary', interpolation=None, aspect='auto')